In [1]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import pandas as pd
import numpy as np
from time import sleep
import h5py
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
# cross_validation deprecated and replaced by the model_selection module  
from sklearn.cross_validation import train_test_split


Using Theano backend.
/home/hd_songm/DrugAI/env/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
#seed for random number reproduction
np.random.seed(2017)

#ngrams range
#minimum range
mi=3
#maximum range
mx=3
#epoch
epoch=20
#batch size
bs=15
#number of cross validation
ncv=5


In [7]:
# read file into pandas from the working directory
data = pd.read_csv('/home/hd_songm/DrugAI/stahl-dataset.csv')
# Reindex the pandas dataframe
data = data.reindex(np.random.permutation(data.index))
print data.shape # dataframe shape

# Smiles strings
X=data.SMILES 
print X.head()

# Target Y indicate whether an inhibitor of "cox2, estrogen, gelatinase  neuramidase  kinase  thrombin  none" or not
Y=data.ix[:,1:7]
Y=Y.values
Y=Y.astype('int')
#print type(Y)

(335, 8)
21     N[S](=O)(=O)C1=CC=C(C=C1)C2=CC(=N[N]2C3=CC=C(F...
30     C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C=C(F)C(=C2)F)C3...
102    C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C(=C)C2)C3=CC=CC=C3
250    CCC(CC)NC1=NC(=CC=N1)C2=C(N=C[N]2C3CC[NH](C)CC...
299         CCOC1=C(Cl)C2=C(C=C(C=C2)[NH+]=C(N)N)C(=O)O1
Name: SMILES, dtype: object


In [8]:
#empty variables
cvscore=[]
lscore=[]
auroc=[]

In [9]:
#custom made k fold cv
#kfold cross validation loop
for i in range(0,ncv):
	
    #spliting data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3, random_state=i)
	
    # instantiate the vectorizer    
    vect = CountVectorizer(analyzer=u'char',lowercase=False,ngram_range=(mi,mx))
	
    # learn training data vocabulary, then use it to create a document-term matrix
    X_train_dtm = vect.fit_transform(X_train)

    # creating model
    model = Sequential()
    model.add(Dense(X_train_dtm.shape[1], input_dim=X_train_dtm.shape[1], init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(150, init='normal', activation='relu'))
    model.add(Dense(y_train.shape[1],init='normal', activation='softmax'))

    #compiling model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])



    # transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm = vect.transform(X_test)
    X_test_dtm

    # Fit the model
    history=model.fit(X_train_dtm.toarray(), y_train,validation_data=(X_test_dtm.toarray(),y_test), shuffle=True, nb_epoch=epoch, batch_size=bs)

    # make class predictions for X_test_dtm
    #y_pred_class = model.predict(X_test_dtm.toarray())

    #np.argmax(y_pred_class)
    #vect.vocabulary_

    #ROC_AUC metrics calculation
    pred_prob = model.predict(X_test_dtm.toarray())
    preds = np.zeros_like(pred_prob)
    preds[np.arange(pred_prob.shape[0]), np.argmax(pred_prob, axis=1)] = 1
    
    iarc=roc_auc_score(y_test,preds)
	
    #accuracy,loss calculation
    score=np.mean(np.array(history.history['val_acc']))
    los=np.mean(np.array(history.history['val_loss']))
    cvscore=np.append(score,cvscore)
    lscore=np.append(los,lscore)
    auroc=np.append(auroc,iarc)
    
    print("kfold: " + str(i+1) + " WAIT..." )
    
    #Prevents "I/O opertion on closed file" value error by adding sleep function
    sleep(5)

Train on 234 samples, validate on 101 samples
Epoch 1/20
234/234 [==============================] - 0s - loss: 1.5195 - acc: 0.4017 - val_loss: 1.3592 - val_acc: 0.4653
Epoch 2/20
234/234 [==============================] - 0s - loss: 0.9572 - acc: 0.6838 - val_loss: 0.8251 - val_acc: 0.6931
Epoch 3/20
234/234 [==============================] - 0s - loss: 0.4947 - acc: 0.8376 - val_loss: 0.5393 - val_acc: 0.8020
Epoch 4/20
234/234 [==============================] - 0s - loss: 0.2436 - acc: 0.9231 - val_loss: 0.4722 - val_acc: 0.8317
Epoch 5/20
234/234 [==============================] - 0s - loss: 0.1045 - acc: 0.9744 - val_loss: 0.3323 - val_acc: 0.8911
Epoch 6/20
234/234 [==============================] - 0s - loss: 0.0544 - acc: 0.9915 - val_loss: 0.2989 - val_acc: 0.8911
Epoch 7/20
234/234 [==============================] - 0s - loss: 0.0245 - acc: 0.9957 - val_loss: 0.4333 - val_acc: 0.8911
Epoch 8/20
234/234 [==============================] - 0s - loss: 0.0167 - acc: 0.9957 - val_l

In [10]:
#Saving models as h5 files
nam="Smiles_model" + str(mi) + "-" + str(mx) + ".h5"
fpath="/home/hd_songm/DrugAI/" + str(nam)
print(fpath)
model.save(fpath)

#Mean calculation
avg=np.mean(cvscore)
lavg=np.mean(lscore)
avg_auroc=np.mean(auroc)

print("\nCV Average Accuracy: "+ str(avg) +"\nCV Average Loss: " + str(lavg) + "\nAUROC: " + str(avg_auroc))

#summary of the model
print(model.summary())

/home/hd_songm/DrugAI/Smiles_model3-3.h5

CV Average Accuracy: 0.862772284649
CV Average Loss: 0.465214689361
AUROC: 0.930824930996
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_37 (Dense)                 (None, 450)           202950      dense_input_10[0][0]             
____________________________________________________________________________________________________
dense_38 (Dense)                 (None, 100)           45100       dense_37[0][0]                   
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 100)           0           dense_38[0][0]                   
____________________________________________________________________________________________________
dense_39 (Dense)                 (None, 150)           15150